The following program was taken from:
https://pyswarms.readthedocs.io/en/latest/examples/usecases/train_neural_network.html

In [10]:
pip install pyswarms

In [11]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
data = load_iris()

# Store the features as X and the labels as y
X = data.data
y = data.target

In [13]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

# Forward propagation
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """

    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [14]:
def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [15]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2023-11-07 02:35:51,749 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.0324
2023-11-07 02:36:34,041 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.03237557898646834, best pos: [ 1.51796280e-01  3.69025130e-01 -4.34012287e-01  4.20758508e-02
  1.16514719e+00 -7.87229850e-01 -1.05322951e+00  8.15825556e-02
  6.58181586e-01  8.04952331e-01  2.02620208e-01  2.36433604e-01
 -2.78033580e-01  1.86218069e+00 -8.73675425e-02  1.36915229e+00
 -4.67028009e-01  1.26933779e-01 -4.08407656e-01  1.41655219e+00
 -9.76727345e-01 -2.25836143e-01 -4.96578400e-02 -1.27955915e-01
 -6.71056955e-02  1.15060756e+00 -5.03777837e-02 -8.42704473e-02
  6.94787367e-01  5.27206910e-01 -7.30342221e-01 -4.15569645e-01
 -5.95471579e-01 -1.79260944e+00  6.82424940e+01 -8.59180991e-02
  6.79804885e-01 -3.32243860e-01  1.22313276e+00 -9.06170324e-01
  6.22281220e-01 -1.481

CPU times: user 36.5 s, sys: 384 ms, total: 36.9 s
Wall time: 42.3 s


In [16]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred


In [17]:
(predict(pos) == y).mean()

0.9866666666666667